# MWE $p_{\parallel}$ analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import ppar as ppar

In [ ]:
INCOMING = '84Ge'
OUTGOING = '83Ge'

#Monte Carlo samples for
#subtraction of bg from peak
MC_NUM   = int(1e4)

#Gate energies
ENERGIES = np.array([#{'peak': 631,'bg': 675},
                     {'peak':1045,'bg':1080},
                     #{'peak':1238,'bg':1290},
                     #{'peak':1359,'bg':1410},
                     {'peak':2113,'bg':2245},
                    ])

#Definition of beam, target, and outgoing fragment.
#This will mainly be needed later for automated
#stopping calculations using Atima.
beam     = {'name':'84Ge','Z':32,'A':52,'mass':83.937575}
product  = {'name':'83Ge','Z':32,'A':51,'mass':82.934539}

target   = {'name': '9Be','Z': 4,'A': 9,'mass': 9.0121821,
            'thickness':1108.8,'density':1.848}

#for plotting
Dic_colors  = {0:'black',1:'firebrick',2:'orange',3:'royalblue',4:'forestgreen'}
Dic_markers = {0:'v',1:'x',2:'.',3:'s',4:'d'}

## Data preparation

Histograms for unreacted and reaction setting are loaded from *.root* files.
Data is extracted and plotted. Momentum distributions from background cuts
are subtracted from cuts on depopulating transitions.
For the sake of simplicty, equal $\gamma$-ray detection efficiencies are assumed.

### Data

In [ ]:
hists       = {key:{} for key in [INCOMING,OUTGOING]}

for key in hists.keys():

    file = 'hist_ppar_in%s_out%s.root'% (INCOMING,key)

    #inclusive distribution
    hists[key]['incl'] = ppar.load_file(file,'ppar_beta/ppar_rest_beta;1',mc_num=MC_NUM)

    #select reacted setting
    if key == INCOMING:
        continue

    for comb in ENERGIES:
        for en in comb.values():

            hists[key][en] = ppar.load_file(file,'ppar_beta/ppar_rest_beta_%i;1'% en,mc_num=MC_NUM)

In [ ]:
for key in hists.keys():
    for name,hist in hists[key].items():

        name = '\ninclusive' if name == 'incl' else '\n%i keV'% name
        name = '(%s,%s)'% (ppar.name_nucl(INCOMING),ppar.name_nucl(key)) + name

        fig,ax = ppar.plot_hist(hist,label=name)

        ax.set_xlim(-2000,2000);

        #ax.set_xlabel(r'$p_{\parallel}$ (MeV/c)', fontsize=16);

### Efficiency

In [ ]:
for comb in ENERGIES:
    for en in comb.values():

        #This must be replaced by real efficiencies!
        eff = np.ones(MC_NUM)

        hists[OUTGOING][en]['values']    = hists[OUTGOING][en]['values']/np.mean(eff)
        hists[OUTGOING][en]['mc_values'] = hists[OUTGOING][en]['mc_values']/eff

### Background subtraction

In [ ]:
hists_corr = {}

for comb in ENERGIES:

    values    = hists[OUTGOING][comb['peak']]['values']    - hists[OUTGOING][comb['bg']]['values']
    mc_values = hists[OUTGOING][comb['peak']]['mc_values'] - hists[OUTGOING][comb['bg']]['mc_values']

    hists_corr[comb['peak']] = {'x_edges':  hists[OUTGOING][comb['peak']]['x_edges'],
                                'x_centers':hists[OUTGOING][comb['peak']]['x_centers'],
                                'values':   values,
                                'mc_values':mc_values,
                               }

In [ ]:
for en in ENERGIES:

    en = en['peak']

    fig,ax = ppar.plot_hist(hists_corr[en],label='%i keV'% en)

    #add uncertainty bands
    plot_mean = np.mean(hists_corr[en]['mc_values'],axis=1)
    plot_std  = np.std(hists_corr[en]['mc_values'],axis=1)

    ax.fill_between(hists_corr[en]['x_centers'],
                    plot_mean-plot_std,plot_mean+plot_std,
                    step='mid',color='grey',alpha=0.5
                   )

    ax.set_xlim(-2000,2000);

    ax.set_xlabel(r'$p_{\parallel}$ (MeV/c)', fontsize=16);

## Theory

Initialization of *ppar* with details on beam, target, and outgoing fragment.
This information will in the future be used for stopping calculations 
(similar to the NSCL part of the *ppar* code).
For now the momentum difference due to the different stopping of beam and fragment
in the target has to be calculated manually using LISE++.

Inclusive parallel momentum distributions from unreacted and reaction settings 
can be loaded and fitted to center them around zero.
A variety of fit models is available; they are selected through the number of input parameters.
Possible are a simple Gaussian (3 parameters), two error functions (4),
two error funtions with an exponential low-momentum tail (7) or
two error funtions with a low-momentum tail comprising an exponential function and a Gaussian (10).

Convolution of parallel momentum distributions obtained from theory with a rectangular function
accounting for the momentum difference due to the uncertain reaction position and the
profile from the unreacted beam setting.

In [ ]:
ppar_mwe = ppar.ppar_RIBF(beam,target,product,verbose=True,)

Calculate the momentum difference due to the unknown reaction position along the target with LISE++
and insert the symmetrized difference, i.e. in the form [-a,a] here.
In the future this might be done automatically using more comprehensive Atima calculations.

In [ ]:
p_diff = [-100,100] #example value

ppar_mwe.calc_stopping(p_diff)

### Unreacted

In [ ]:
ppar_mwe.load_unreacted(file='hist_ppar_in%s_out%s.root'% (INCOMING,INCOMING),
                        histogram='ppar_beta/ppar_rest_beta;1',
                        rebin=1
                       )

ppar_mwe.plot_unreacted(xlim=[-1000,1000]);

In [ ]:
x0 = [1e8,0,10]
#x0 = [1e8,0,10,10]

ppar_mwe.fit_unreacted(fit_range=[-250,250],x0=x0)
ppar_mwe.plot_unreacted(xlim=[-1000,1000]);

### Reaction setting

In [ ]:
ppar_mwe.load_reacted(file='hist_ppar_in%s_out%s.root'% (INCOMING,OUTGOING),
                      histogram='ppar_beta/ppar_rest_beta;1',
                      rebin=1
                     )

ppar_mwe.plot_reacted(xlim=[-1000,1000]);

In [ ]:
#x0 = [1e8,0,10]
x0 = [1e8,0,10,10]
#x0 = [0,0,0,1e8,0,10,10]

ppar_mwe.fit_reacted(fit_range=[-250,250],x0=x0)
ppar_mwe.plot_reacted(xlim=[-1000,1000]);

### Comparison

In [ ]:
fig,ax = ppar_mwe.plot_unreacted(plot_fit=False,xlim=[-1000,1000],label='Unreacted');

x_plot = ppar_mwe.spec_reac['x_centers']
y_plot = ppar_mwe.spec_reac['values']
scale  = np.sum(ppar_mwe.spec_unreac['values'])/np.sum(ppar_mwe.spec_reac['values'])

ax.step(x_plot,scale*y_plot,where='mid',color='forestgreen',label='Reacted')

plt.legend(fontsize=16)

### Convolution of theoretical distributions

In [ ]:
theo_mwe = {name:ppar.state(ppar_mwe,'ppar_theo/pparGe83_%s.dat'% name) for name in ['3s1','2d5','1g9']}

#### $3s_{1/2}$

In [ ]:
ORB = '3s1'

theo_mwe[ORB].plot_theory();
theo_mwe[ORB].convolve_theory();
theo_mwe[ORB].plot_theory();

#### $2d_{5/2}$

In [ ]:
ORB = '2d5'

theo_mwe[ORB].plot_theory();
theo_mwe[ORB].convolve_theory();
theo_mwe[ORB].plot_theory();

#### $1g_{9/2}$

In [ ]:
ORB = '1g9'

theo_mwe[ORB].plot_theory();
theo_mwe[ORB].convolve_theory();
theo_mwe[ORB].plot_theory();

## Comparison between theory and data

Comparison of background-subtracted, experimental parallel momentum distributions to
convolved distributions from theory. 

The experimental distribution can be rebinned if needed and (almost) empty bins can be cut
with the *threshold* argument. The theoretical distribution is scaled to the maximum bin content
of the experimental distribution using the *scale_theory* method.

If possible the *plot_hist* method will include the scaled theoretical distribution.
This behavior can be suppressed by setting the *plot_theory* argument to *False*.
The plots can be somewhat modified using the appropriate keyword arguments.
For more customization the *figure* and *axes* objects are returned.

### 1359 keV

In [ ]:
EN = 1045

#Histogram needs to be redefined: mc_vales => values
hist = {'x_centers':hists_corr[EN]['x_centers'],
        'x_edges':  hists_corr[EN]['x_edges'],
        'values':   hists_corr[EN]['mc_values'],
       }

for orb in theo_mwe.keys():

    theo_mwe[orb].rebin_hist(hist,
                             rebin=2,
                             threshold=10,
                             )
    #theo_mwe[orb].plot_hist(xlim=[-1000,1000],ylim=[-10,600]);

    theo_mwe[orb].scale_theory()
    theo_mwe[orb].plot_hist(xlim=[-1000,1000],ylim=[-10,600]);

In [ ]:
#Plot of experimental parallel momentum distribution with all distributions from theory.
fig,ax = theo_mwe['3s1'].plot_hist(plot_theory=False,xlim=[-1000,1000],ylim=[-10,600]);

for num,orb in enumerate(theo_mwe.keys()):

    x_plot = theo_mwe[orb].ppar_theo['tail_target']['x_centers']
    y_plot = theo_mwe[orb].ppar_theo['tail_target']['values']
    scale  = theo_mwe[orb].fit_res
    
    ax.plot(x_plot,scale*y_plot,color=Dic_colors[num],label=orb)

    ax.legend(fontsize=16)

### 2113 keV

In [ ]:
EN = 2113

#Histogram needs to be redefined: mc_vales => values
hist = {'x_centers':hists_corr[EN]['x_centers'],
        'x_edges':  hists_corr[EN]['x_edges'],
        'values':   hists_corr[EN]['mc_values'],
       }

for orb in theo_mwe.keys():

    theo_mwe[orb].rebin_hist(hist,
                             rebin=1,
                             threshold=10,
                             )
    #theo_mwe[ORB].plot_hist(xlim=[-1000,1000],ylim=[-10,600]);

    theo_mwe[orb].scale_theory()
    theo_mwe[orb].plot_hist(xlim=[-1000,1000],ylim=[-10,600]);

In [ ]:
#Plot of experimental parallel momentum distribution with all distributions from theory.
fig,ax = theo_mwe['3s1'].plot_hist(plot_theory=False,xlim=[-1000,1000],ylim=[-10,600]);

for num,orb in enumerate(theo_mwe.keys()):

    x_plot = theo_mwe[orb].ppar_theo['tail_target']['x_centers']
    y_plot = theo_mwe[orb].ppar_theo['tail_target']['values']
    scale  = theo_mwe[orb].fit_res
    
    ax.plot(x_plot,scale*y_plot,color=Dic_colors[num],label=orb)

    ax.legend(fontsize=16)